In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import HuberRegressor

# Diagram settings
title_font_size = 12
axis_font_size = 12

# For reproducibility
np.random.seed(42)

def plot_description(text):
    print(f"\nDescription:\n{text}\n")

In [2]:
# Visualize loss functions: MSE vs. MAE vs. Huber
def loss_functions_visualize():    
    # ---------------------------------------------------
    # 1. Visualize MSE, MAE, and Huber Loss
    # ---------------------------------------------------
    errors = np.linspace(-2, 2, 200)
    delta = 1.0  # Huber threshold
    
    mse = errors**2
    mae = np.abs(errors)
    huber = np.where(np.abs(errors) <= delta,
                     0.5 * errors**2,
                     delta * (np.abs(errors) - 0.5 * delta))
    
    plt.figure(figsize=(7, 5))
    plt.plot(errors, mse, label="MSE", linewidth=2)
    plt.plot(errors, mae, label="MAE", linewidth=2)
    plt.plot(errors, huber, label="Huber (δ=1)", linewidth=2)
    plt.title("Loss Functions vs. Prediction Error", fontsize=title_font_size)
    plt.xlabel("Prediction Error (y - ŷ)", fontsize=axis_font_size)
    plt.ylabel("Loss", fontsize=axis_font_size)
    plt.legend()
    plt.grid(True)
    plt.show()
    

In [3]:
from ipywidgets import interactive, FloatSlider, IntSlider, VBox, Label, Layout

def linear_reg_loss_demo(num_outliers=50):
    # ---------------------------------------------------
    # 1. Create a simple regression dataset
    # ---------------------------------------------------
    n_samples = 500
    X, y = make_regression(n_samples=n_samples, n_features=1, noise=15, random_state=42)

    # Add outliers controlled by sliders
    outlier_indices = np.random.choice(n_samples, num_outliers, replace=False)
    y[outlier_indices] += 200 * np.random.randn(num_outliers)

    # ---------------------------------------------------
    # 2. Train models with different loss functions
    # ---------------------------------------------------
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    ols = LinearRegression().fit(X_scaled, y)

    sgd_mae = SGDRegressor(loss='epsilon_insensitive', epsilon=0.0, 
                           max_iter=10000, tol=1e-3, random_state=42).fit(X_scaled, y)
    
    huber_reg = HuberRegressor(epsilon=1.0, max_iter=1000).fit(X_scaled, y)
    
    # Predictions
    x_line = np.linspace(X_scaled.min(), X_scaled.max(), 100).reshape(-1, 1)
    y_ols = ols.predict(x_line)
    y_mae = sgd_mae.predict(x_line)
    y_huber = huber_reg.predict(x_line)
    
    # ---------------------------------------------------
    # 3. Plot results
    # ---------------------------------------------------
    plt.figure(figsize=(8, 6))
    plt.scatter(X_scaled, y, color='gray', alpha=0.4, label="Data")
    plt.plot(x_line, y_ols, label="MSE", linewidth=2)
    plt.plot(x_line, y_mae, label="MAE", linewidth=2)
    plt.plot(x_line, y_huber, label="Huber", linewidth=2)
    plt.title("Effect of Different Loss Functions on Fit", fontsize=title_font_size)
    plt.xlabel("X", fontsize=axis_font_size)
    plt.ylabel("y", fontsize=axis_font_size)
    plt.legend()
    plt.grid(True)
    plt.show()


# ---------------------------------------------------
    # 4. Create clearly labeled, wide sliders inside a titled block
    # ---------------------------------------------------
def linear_reg_loss_functions_demo_interact():
    plot_description("Three linear models using different loss functions (MSE/MAE/Huber) are fitted to the samples (gray dots). "
                     "Increase the number of outliers in the dataset and notice differences in the models. "
                     "Takeaway: Model predictions vary based on the loss function that was used to train the model.")
    
    num_outliers_slider = IntSlider(
        value=50, min=0, max=500, step=5,
        description="Number of Outliers",
        style={'description_width': '150px'},
        layout=Layout(width='500px')
    )

    ui_box = VBox([
        Label(value="📊 Controls", layout=Layout(margin="0 0 0 0")),
    ])

    interactive_plot = interactive(
        linear_reg_loss_demo,
        num_outliers=num_outliers_slider
    )

    display(ui_box, interactive_plot)

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, SGDRegressor, HuberRegressor
from ipywidgets import interactive, FloatSlider, IntSlider, VBox, Label, Layout
from IPython.display import display

def polynomial_regression_loss_demo(num_outliers=50, poly_estimator_degree=3):
    # ---------------------------------------------------
    # 1. Create nonlinear dataset (quadratic + noise)
    # ---------------------------------------------------
    np.random.seed(42)
    n_samples = 400
    X = np.linspace(-3, 3, n_samples).reshape(-1, 1)
    y = 3 * X.squeeze()**3 - 3 * X.squeeze()**2 - 4 * X.squeeze() + np.random.randn(n_samples) * 2

    # Add outliers
    outlier_indices = np.random.choice(n_samples, num_outliers, replace=False)
    y[outlier_indices] += 30 * np.random.randn(num_outliers)
    
    # Polynomial features
    poly = PolynomialFeatures(degree=poly_estimator_degree, include_bias=False)
    X_poly = poly.fit_transform(X)
    
    # Normalize for fairness
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_poly)
    
    # ---------------------------------------------------
    # 2. Train models with different loss functions
    # ---------------------------------------------------
    ols = LinearRegression().fit(X_scaled, y)
    sgd_mae = SGDRegressor(loss='epsilon_insensitive', epsilon=0.0, 
                           max_iter=10000, tol=1e-3, random_state=42).fit(X_scaled, y)
    huber_reg = HuberRegressor(epsilon=1.0, max_iter=1000).fit(X_scaled, y)
    
    # ---------------------------------------------------
    # 3. Predict on dense grid for smooth curves
    # ---------------------------------------------------
    X_line = np.linspace(X.min(), X.max(), 200).reshape(-1, 1)
    X_line_poly = poly.transform(X_line)
    X_line_scaled = scaler.transform(X_line_poly)
    
    y_ols = ols.predict(X_line_scaled)
    y_mae = sgd_mae.predict(X_line_scaled)
    y_huber = huber_reg.predict(X_line_scaled)
    
    # ---------------------------------------------------
    # 4. Visualization
    # ---------------------------------------------------
    plt.figure(figsize=(8, 6))
    plt.scatter(X, y, color='gray', alpha=0.4, label="Data")
    plt.plot(X_line, y_ols, label="MSE", linewidth=2)
    plt.plot(X_line, y_mae, label="MAE", linewidth=2)
    plt.plot(X_line, y_huber, label="Huber", linewidth=2)
    
    plt.title("Polynomial Regression with Different Loss Functions", fontsize=16)
    plt.xlabel("X", fontsize=14)
    plt.ylabel("y", fontsize=14)
    plt.legend()
    plt.grid(True)
    plt.show()


# ---------------------------------------------------
# 5. Interactive control panel (same style as before)
# ---------------------------------------------------
def polynomial_regression_loss_demo_interact():
    plot_description("Three polynomial models (with MSE/MAE/Huber loss) are fitted to the data. Play around with the"
                     " number of outliers and the polynomial degree of the models. Notice differences in the models.")
    
    num_outliers_slider = IntSlider(
        value=10, min=0, max=200, step=5,
        description="Number of Outliers",
        style={'description_width': '150px'},
        layout=Layout(width='500px')
    )

    poly_estimator_degree = IntSlider(
        value=3, min=1, max=6, step=1,
        description="Polyn. Estimator Degree",
        style={'description_width': '150px'},
        layout=Layout(width='500px')
    )

    ui_box = VBox([
        Label(value="📊 Controls", layout=Layout(margin="0 0 0 0")),
    ])

    interactive_plot = interactive(
        polynomial_regression_loss_demo,
        num_outliers=num_outliers_slider,
        poly_estimator_degree=poly_estimator_degree
    )

    display(ui_box, interactive_plot)


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from ipywidgets import interactive, IntSlider, FloatSlider, VBox, Label, Layout
from IPython.display import display

def polynomial_overfitting_demo(poly_degree=1, noise_level=10.0, num_samples=60):
    # ---------------------------------------------------
    # 1. Create synthetic data from a known nonlinear function
    # ---------------------------------------------------
    np.random.seed(42)
    X = np.linspace(-3, 3, num_samples).reshape(-1, 1)
    y_true = np.sin(X).ravel()
    
    # Add Gaussian noise
    y = y_true + np.random.randn(num_samples) * (noise_level / 10)
    
    # ---------------------------------------------------
    # 2. Build and fit polynomial regression model
    # ---------------------------------------------------
    poly = PolynomialFeatures(degree=poly_degree, include_bias=False)
    X_poly = poly.fit_transform(X)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_poly)
    
    model = LinearRegression().fit(X_scaled, y)
    
    # ---------------------------------------------------
    # 3. Generate smooth prediction curve
    # ---------------------------------------------------
    X_line = np.linspace(-3, 3, 300).reshape(-1, 1)
    X_line_poly = poly.transform(X_line)
    X_line_scaled = scaler.transform(X_line_poly)
    y_pred = model.predict(X_line_scaled)
    
    # ---------------------------------------------------
    # 4. Plot results
    # ---------------------------------------------------
    plt.figure(figsize=(8, 6))
    plt.scatter(X, y, color="gray", alpha=0.7, label="Training Data")
    plt.plot(X_line, np.sin(X_line), color="green", linewidth=2, label="True Function (sin x)")
    plt.plot(X_line, y_pred, color="red", linewidth=2, label=f"Polynomial Fit (degree={poly_degree})")
    
    plt.title("Effect of Model Complexity (Overfitting Demonstration)", fontsize=16)
    plt.xlabel("X", fontsize=14)
    plt.ylabel("y", fontsize=14)
    plt.legend()
    plt.grid(True)
    plt.show()


# ---------------------------------------------------
# 5. Interactive controls
# ---------------------------------------------------
def polynomial_overfitting_demo_interact():
    plot_description("Demonstration of overfitting with a polynomial model. The true function to be fitted is a sinus curve (green line)."
                     " The samples (gray dots) are the (noisy) measurements taken from the true function. The task here is to "
                     "create a polynomial model (red line) that resembles the true function as closely as possible, based on the measured"
                     " samples. Increase the polynomial degree of the model and notice how the model is overfitting the samples "
                    "starting from degree > 4.")
    
    degree_slider = IntSlider(
        value=1, min=1, max=20, step=1,
        description="Polynomial Degree",
        style={'description_width': '150px'},
        layout=Layout(width='500px')
    )
    noise_slider = FloatSlider(
        value=10.0, min=0.0, max=50.0, step=2.0,
        description="Noise Level",
        style={'description_width': '150px'},
        layout=Layout(width='500px')
    )
    sample_slider = IntSlider(
        value=60, min=20, max=200, step=10,
        description="Number of Samples",
        style={'description_width': '150px'},
        layout=Layout(width='500px')
    )

    ui_box = VBox([
        Label(value="📊 Controls", layout=Layout(margin="0 0 0 0")),
    ])

    interactive_plot = interactive(
        polynomial_overfitting_demo,
        poly_degree=degree_slider,
        noise_level=noise_slider,
        num_samples=sample_slider
    )

    display(ui_box, interactive_plot)


In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from ipywidgets import interactive, FloatSlider, VBox, Label, Layout
from IPython.display import display

def binary_classification_demo(separation=1.0):
    # ---------------------------------------------------
    # 1. Create binary dataset
    # ---------------------------------------------------
    np.random.seed(42)
    X, y = make_classification(
        n_samples=200, n_features=2, n_informative=2, n_redundant=0,
        n_clusters_per_class=1, class_sep=separation, random_state=42
    )
    
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # ---------------------------------------------------
    # 2. Train logistic regression model
    # ---------------------------------------------------
    model = LogisticRegression()
    model.fit(X, y)
    
    # ---------------------------------------------------
    # 3. Visualize decision boundary
    # ---------------------------------------------------
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200))
    Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, 1-Z, cmap="RdYlBu", alpha=0.5)
    plt.scatter(X[y==0, 0], X[y==0, 1], color="blue", label="Class 0", edgecolor="black", linewidth=0.5)
    plt.scatter(X[y==1, 0], X[y==1, 1], color="red", label="Class 1", edgecolor="black", linewidth=0.5)
    plt.title("Binary Classification with Logistic Regression", fontsize=16)
    plt.xlabel("Feature 1", fontsize=14)
    plt.ylabel("Feature 2", fontsize=14)
    plt.legend(title="Ground Truth")
    plt.grid(True)
    plt.show()

def binary_classification_demo_interact():
    # ---------------------------------------------------
    # 4. Visualize Binary Cross-Entropy loss function
    # ---------------------------------------------------
    p = np.linspace(0.001, 0.999, 200)
    loss_positive = -np.log(p)
    loss_negative = -np.log(1 - p)

    plot_description("We are looking at a binary classification problem (i.e., assigning a sample to one of two classes). The Figure"
                     " below shows an example visualization of the Binary Cross-Entropy (BCE) loss function that was used to train the "
                     "logistic regression model. The BCE function calculates the loss values based on the orange line (for samples of class 0) "
                    "or based on the blue line (for samples of class 1).")
    
    plt.figure(figsize=(6, 4))
    plt.plot(p, loss_positive, label="Loss (True Label = 1)", linewidth=2)
    plt.plot(p, loss_negative, label="Loss (True Label = 0)", linewidth=2)
    plt.title("Binary Cross-Entropy Loss", fontsize=14)
    plt.xlabel("Predicted Probability (p)", fontsize=12)
    plt.ylabel("Loss", fontsize=12)
    plt.legend()
    plt.grid(True)
    plt.show()

    plot_description("Looking at the plot below, the blue and rot dots represent training samples of class 0 and class 1 respectively."
                     " The background shows prediction probabilities of the logisitic regression model."
                     " Bluish background: High probability for Class 0. Reddish background: High probability for Class 1. \n"
                     "The model assigns class 0 or 1 to a new \"unseen\" sample, based on these probabilities. "
                     "Notice how prediction uncertainty increases as training samples of both classes get mixed (play around with the slider below).")
    
    # --- Interactive control ---
    sep_slider = FloatSlider(
        value=1.0, min=0.1, max=3.0, step=0.1,
        description="Class Separation",
        style={'description_width': '150px'},
        layout=Layout(width='500px'),
    )

    ui_box = VBox([
        Label(value="📊 Controls", layout=Layout(margin="0 0 0 0")),
    ])
    interactive_plot = interactive(binary_classification_demo, separation=sep_slider)

    display(ui_box, interactive_plot)

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from ipywidgets import interactive, FloatSlider, VBox, Label, Layout
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

def multi_class_classification_demo(separation=1.0):
    # ---------------------------------------------------
    # 1. Create multi-class dataset
    # ---------------------------------------------------
    np.random.seed(42)
    X, y = make_classification(
        n_samples=300, n_features=2, n_informative=2, n_redundant=0,
        n_classes=3, n_clusters_per_class=1, class_sep=separation, random_state=42
    )
    
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # ---------------------------------------------------
    # 2. Train logistic regression model for multi-class
    # ---------------------------------------------------
    model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
    model.fit(X, y)
    
    # ---------------------------------------------------
    # 3. Visualize decision boundaries
    # ---------------------------------------------------
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200))
    Z_probabilities = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])
        
    Z_max_proba = Z_probabilities.max(axis=1)
    Z_max_proba = Z_max_proba.reshape(xx.shape)
    
    Z_predictions = np.argmax(Z_probabilities, axis=1)
    Z_predictions = Z_predictions.reshape(xx.shape)
    
    # Plot
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    plt.suptitle("Multi-Class Classification with Logistic Regression", fontsize=16)
    titles = ["Max. Prediction Probabilities", "Model Decision Boundaries"]
    
    for i, title in enumerate(titles):
        if i == 0:
            zz = Z_max_proba
            cmap = "Blues"
        else:
            zz = Z_predictions
            cmap = "viridis"

        ax[i].set_title(title, fontsize=12)
        ax[i].contourf(xx, yy, zz, cmap=cmap , alpha=0.5)
        ax[i].scatter(X[y==0, 0], X[y==0, 1], color="blue", label="Class 0", edgecolor="black", linewidth=0.5)
        ax[i].scatter(X[y==1, 0], X[y==1, 1], color="red", label="Class 1", edgecolor="black", linewidth=0.5)
        ax[i].scatter(X[y==2, 0], X[y==2, 1], color="green", label="Class 2", edgecolor="black", linewidth=0.5)
        ax[i].set_xlabel("Feature 1", fontsize=12)
        ax[i].set_ylabel("Feature 2", fontsize=12)
        ax[i].legend(title="Ground Truth")
        ax[i].grid(True)
    
    plt.show()

def multi_class_classification_demo_interact():
    plot_description("Example for Multi-Class Classification with a logistic regression model trained using the cross-entropy loss function."
                     " Training samples are shown in blue (class 0), red (class 1) and green (class 2). The model outputs three probability values,"
                     " one for each class, based on the sample's input features (Feature 1 & 2).\n\n"
                     "Left plot: The colored background shows the maximum probability value for each combination of Feature 1 & 2. The darker the "
                     "background, the higher the probability. Notice how the light area (i.e., the model's uncertainty) increases as training samples"
                     " of different classes mix (play around with the slider below).\n\n"
                     "Right plot: The background colors of this plot show the model decisions (i.e., assignment of input feature combinations to one class)."
                     " Purple area => Class 0, Turquoise area => Class 1, Yellow area => Class 2.")
    
    # --- Interactive control ---
    sep_slider = FloatSlider(
        value=1.0, min=0.1, max=3.0, step=0.1,
        description="Class Separation",
        style={'description_width': '150px'},
        layout=Layout(width='500px')
    )
    ui_box = VBox([
        Label(value="📊 Controls", layout=Layout(margin="0 0 0 0")),
    ])
    interactive_plot = interactive(multi_class_classification_demo, separation=sep_slider)
    display(ui_box, interactive_plot)

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from ipywidgets import interact, FloatSlider, VBox, HTML

!pip install scikit-learn-extra
from sklearn_extra.cluster import KMedoids
from sklearn_extra.cluster import KMedoids


def clustering_demo(class_separation=0.0):
    np.random.seed(0)
    
    # Generate two rectangular clusters
    cluster1 = np.random.uniform(low=[0, 0], high=[4, 4], size=(200, 2))
    cluster2 = np.random.uniform(low=[4 - class_separation, 0],
                                 high=[6 - class_separation, 4], size=(200, 2))
    X = np.vstack([cluster1, cluster2])
    
    # Fit models
    kmeans = KMeans(n_clusters=2, n_init=10, random_state=0).fit(X)
    kmedians = KMedoids(n_clusters=2, metric='manhattan', random_state=0).fit(X)
    
    # Sort clusters by x-coordinate of their centers (to ensure consistent colors)
    def sort_clusters(model):
        centers = model.cluster_centers_
        order = np.argsort(centers[:, 0])
        labels_sorted = np.zeros_like(model.labels_)
        for new_label, old_label in enumerate(order):
            labels_sorted[model.labels_ == old_label] = new_label
        return labels_sorted, centers[order]
    
    labels_kmeans, centers_kmeans = sort_clusters(kmeans)
    labels_kmedians, centers_kmedians = sort_clusters(kmedians)
    
    # Plot
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    titles = ["K-Means (L2 loss)", "K-Medians (L1 loss)"]
    
    for i, (labels, centers, title) in enumerate(
        zip([labels_kmeans, labels_kmedians],
            [centers_kmeans, centers_kmedians],
            titles)
    ):
        ax[i].scatter(X[labels==0, 0], X[labels==0, 1], color="blue" if "K-Means" in title else "green", label="Cluster 0", s=40, alpha=0.7)
        ax[i].scatter(X[labels==1, 0], X[labels==1, 1], color="orange" if "K-Means" in title else "purple", label="Cluster 1", s=40, alpha=0.7)
        ax[i].scatter(centers[:, 0], centers[:, 1],
                      c="red", s=200 if "K-Means" in title else 100,
                      marker="X" if "K-Means" in title else "D",
                      edgecolor="black", label="Means" if "K-Means" in title else "Medians")
        ax[i].set_title(title, fontsize=14)
        ax[i].set_xlim(-1, 7)
        ax[i].set_ylim(-1, 6)
        ax[i].grid(True)
        ax[i].legend()
    
    plt.show()

def clustering_demo_interact():
    plot_description("Clustering Demo: K-Means (trained using L2 loss) vs. K-Medians (trained using L1 loss). Note that "
                     "these plots demonstrate an example of unsupervised learning (in contrast to previous plots). "
                     "The task of the K-Means and K-Medians algorithms is to find clusters independent of any pre-defined ground truth labels.\n\n"
                    "Left Plot: Clusters found by K-Means with respective cluster means (red crosses).\n\n"
                    "Right Plot: Clusters found by K-Medians with respective cluster medians (red squares).\n\n"
                    "In this case, we instructed both algorithms to find two distinct clusters. Notice how the identified\n"
                    " clusters differ as the sample distribution changes (play around with the slider below).")
    
    # --- Interactive control ---
    sep_slider2 = FloatSlider(
        value=0.0, min=-1.0, max=4.0, step=0.1,
        description="Move Samples",
        style={'description_width': '150px'},
        layout=Layout(width='500px')
    )
    ui_box = VBox([
        Label(value="📊 Controls", layout=Layout(margin="0 0 0 0")),
    ])
    interactive_plot = interactive(clustering_demo, class_separation=sep_slider2)
    display(ui_box, interactive_plot)

  Using cached scikit_learn_extra-0.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.6 kB)
Using cached scikit_learn_extra-0.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)


In [11]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, FloatSlider, IntSlider
from ipywidgets import interactive, VBox, Output, Label

# --- Define the parabola function ---
def f(x):
    return x**2  # simple convex function

def df(x):
    return 2*x  # derivative

# --- Optimization routine ---
def optimize(optimizer="GD", lr=0.1, epochs=20, momentum=0.9):
    x = -8 # np.random.uniform(-8, 8)  # random start
    v = 0  # momentum buffer
    history = [x]

    for i in range(epochs):
        grad = df(x)

        if optimizer == "GD":
            x = x - lr * grad

        elif optimizer == "Momentum":
            v = momentum * v - lr * grad
            x = x + v

        history.append(x)

    return np.array(history)

# --- Plotting function ---
def train_and_plot(optimizer, lr, epochs, momentum):
    xs = np.linspace(-11, 11, 500)
    ys = f(xs)

    path = optimize(optimizer, lr, epochs, momentum)
    path_y = f(path)

    plt.figure(figsize=(8,6))
    plt.plot(xs, ys, label="f(x) = x²", color="blue", alpha=0.5)
    plt.plot(path, path_y, color="red", linestyle="--", alpha=0.7, zorder=3)
    plt.scatter(path[:-1], path_y[:-1], s=60, label="Steps", marker="o", facecolors='none', edgecolors='black', zorder=4)
    plt.scatter(path[-1], path_y[-1], color="red", s=80, label="Final Step", marker="x", zorder=5)

    plt.title(f"Params: lr={lr:.2f}, epochs={epochs}, momentum={momentum:.2f}" if optimizer=="Momentum" else f"{optimizer}: lr={lr:.2f}, epochs={epochs}")
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.legend()
    plt.xlim([-12.5, 12.5])
    plt.ylim([-5, 105])
    plt.grid(True)
    plt.show()

# --- Widgets ---
# Define widgets with default values
optimizer_widget = Dropdown(options=["GD", "Momentum"], value="GD", description="Optimizer")

# Store defaults for reset
defaults = {
    "lr": 0.05,
    "epochs": 10,
    "momentum": 0.75
}

lr_widget = FloatSlider(value=defaults["lr"], min=0.01, max=1.1, step=0.01, description="Learning rate", readout_format=".2f")
epochs_widget = IntSlider(value=defaults["epochs"], min=5, max=100, step=5, description="Epochs")
momentum_widget = FloatSlider(value=defaults["momentum"], min=0.1, max=0.99, step=0.05, description="Momentum", readout_format=".2f")
momentum_widget.layout.visibility = 'hidden'

def on_optimizer_change(change):
    if change["name"] == "value":
        lr_widget.value = defaults["lr"]
        epochs_widget.value = defaults["epochs"]
        momentum_widget.value = defaults["momentum"]

        if change["new"] == "Momentum":
            momentum_widget.layout.visibility = 'visible'
        else:
            momentum_widget.layout.visibility = 'hidden'

optimizer_widget.observe(on_optimizer_change, names="value")

label = Label(value="📊 Controls", layout=Layout(margin="0 0 8px 0"))

# Add left margin (indent) to the widgets you want to indent
margin_left = 40
optimizer_widget.layout.margin = f"0 0 0 {margin_left}px"
lr_widget.layout.margin = f"0 0 0 {margin_left}px"
epochs_widget.layout.margin = f"0 0 0 {margin_left}px"
momentum_widget.layout.margin = f"0 0 0 {margin_left}px"

ui = VBox([label, optimizer_widget, lr_widget, epochs_widget, momentum_widget])
out = Output()

def wrapped_train_and_plot(optimizer, lr, epochs, momentum):
    with out:
        out.clear_output(wait=True)
        train_and_plot(optimizer, lr, epochs, momentum)

def gradient_descent_interact():
    plot_description("Gradient descent (GD) demonstration. Use the dropdown to switch between standard GD and GD+Momentum."
                    " Adjust learning rate, number of epochs and momentum. Notice how the process diverges when learning rate > 1.")
    
    interactive_plot = interactive(
        wrapped_train_and_plot,
        optimizer=optimizer_widget,
        lr=lr_widget,
        epochs=epochs_widget,
        momentum=momentum_widget,
    )
    
    display(ui, out)
